# 複数の機械学習ライブラリを用いて線形回帰を行う

In [2]:
import os
import numpy as np

import sys
sys.path.append("..")

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

import tensorflow as tf
from tensorflow.python.layers import core as core_layers
try:
    from mpl_toolkits.mplot3d import Axes3D
except: pass

/home/ksn/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 次の2つの回帰式の検証のために人工的に作り出したデータを使う

### 線形回帰

$ y(x) = a + b_1 \cdot X_1 + b_2 \cdot X_2 + b_3 \cdot X_3 + \sigma \cdot \varepsilon $ 

ここで $ \varepsilon \sim N(0, 1) $ はガウシアンノイズ。また、 $ \sigma $ はボラティリティを表す。  
パラメータの値は以下のように値を選ぶ。（この既知のパラメータを推定してみるということ）  

$ a = 1.0 $

$ b_1, b_2, b_3 = (1.0,0.5, 0.2) $

$ \sigma = 0.1 $

$ X_1, X_2, X_3 $ will be uniformally distributed in $ [-1,1] $


### 非線形回帰

$ y(x) = a + w_{00} \cdot X_1 + w_{01} \cdot X_2 + w_{02} \cdot X_3 + w_{10} \cdot X_1^2 
+ w_{11} \cdot X_2^2 + w_{12} \cdot X_3^2 +  \sigma \cdot \varepsilon $ 

ここで

$ w = [[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]]  $

である。また、残りのパラメータは上記の線形回帰のときと同じで、$ X_i $に関しても同様。


### データの生成

In [17]:
def generate_data(n_points=10000, n_features=3, use_nonlinear=True, 
                    noise_std=0.1, train_test_split = 4):
    """
    Arguments:
    n_points - number of data points to generate
    n_features - a positive integer - number of features
    use_nonlinear - if True, generate non-linear data
    train_test_split - an integer - what portion of data to use for testing
                     - テストに使うデータの割合
    
    Return:
    X_train, Y_train, X_test, Y_test, n_train, n_features
    """
    
    # Linear data or non-linear data?
    # もしパラメータuse_nonlinearがTrueであれば非線形モデルを用いる
    # 今回は使っていない
    if use_nonlinear:
        # [[w_00,w_01,w_02],[w_10,w_11,w_12]]
        weights = np.array([[1.0, 0.5, 0.2],[0.5, 0.3, 0.15]])
    else:
        # [a, b_1, b_2]
        weights = np.array([1.0, 0.5, 0.2])
    
    bias = np.ones(n_points).reshape((-1,1))
    low = - np.ones((n_points,n_features),'float') # 一様分布から取り出された確率変数が取る値の下限
    high = np.ones((n_points,n_features),'float')  # 一様分布から取り出された確率変数が取る値の上限
        
    np.random.seed(42)
    X = np.random.uniform(low=low, high=high)
    
    np.random.seed(42)
    noise = np.random.normal(size=(n_points, 1))
    noise_std = 0.1
    
    if use_nonlinear:
        # weights[0,0]とbiasをかけてバイアス項を表すのは本当はよくないとおもわれる
        # 本来weights[0,0]はaを表すのではなくてw_00を表しているので（今、どちらも1で同じ値なので結果は変わらない）
        Y = (weights[0,0] * bias + np.dot(X, weights[0, :]).reshape((-1,1)) + 
             np.dot(X*X, weights[1, :]).reshape([-1,1]) +
             noise_std * noise)
    else:
        Y = (weights[0] * bias + np.dot(X, weights[:]).reshape((-1,1)) + 
             noise_std * noise)
    
    n_test = int(n_points/train_test_split)
    n_train = n_points - n_test
    
    X_train = X[:n_train,:]
    Y_train = Y[:n_train].reshape((-1,1))

    X_test = X[n_train:,:]
    Y_test = Y[n_train:].reshape((-1,1))
    
    return X_train, Y_train, X_test, Y_test, n_train, n_features

In [5]:
X_train, Y_train, X_test, Y_test, n_train, n_features = generate_data(use_nonlinear=False)
X_train.shape, Y_train.shape

((7500, 3), (7500, 1))

### Numpyを使って線形回帰

Normal Equationを使って計算している。

Normal Equationはよく見る形としてはこんな感じ:  
$\theta = (X^T X)^{-1} X ^{T} y$

In [6]:
num_of_f = len(X_train)
X = np.hstack((np.ones(num_of_f).reshape((-1,1)),X_train))
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y_train)

In [7]:
theta_numpy

array([[0.99946227],
       [0.99579039],
       [0.499198  ],
       [0.20019798]])

実際のデータは切片の値が1, b_1=1.0, b_2=0.5, b_3=0.2として生成されたものなのでかなり正確に推定出来ていることが分かる

### Sklearnを使って線形回帰

これは簡単で分かりやすい。scikit-learnを使う場合はモデルを選んで訓練データとテストデータにフィットさせるという流れらしい。

In [8]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
reg = lin_reg.fit(X_train,Y_train)

推定された係数の値は次のように確認できる:

In [9]:
reg.coef_

array([[0.99579039, 0.499198  , 0.20019798]])

切片は以下:

In [10]:
lin_reg.intercept_

array([0.99946227])

### Tensorflowを使って線形回帰

やはりNormal Equationを使って計算している。

Normal Equationはよく見る形としてはこんな感じ:  
$\theta = (X^T X)^{-1} X ^{T} y$

In [14]:
num_of_f = len(X_train)

In [15]:
X_np = np.hstack((np.ones(num_of_f).reshape((-1,1)),X_train))

In [16]:
X_np

array([[ 1.        , -0.25091976,  0.90142861,  0.46398788],
       [ 1.        ,  0.19731697, -0.68796272, -0.68801096],
       [ 1.        , -0.88383278,  0.73235229,  0.20223002],
       ...,
       [ 1.        , -0.41969273,  0.7395476 ,  0.49515624],
       [ 1.        , -0.15112479,  0.42105835,  0.73537775],
       [ 1.        ,  0.95130426,  0.90470301, -0.34934435]])

In [18]:
def tf_lin_regress(X_train, Y_train):
    """
    Arguments:
    X_train  - np.array of size (n by k) where n is number of observations 
                of independent variables and k is number of variables
    Y_train - np.array of size (n by 1) where n is the number of observations of dependend variable
    
    Return:
    np.array of size (k+1 by 1) of regression coefficients
    """
    num_of_f = len(X_train)
    # add the column of ones
    X_np = np.hstack((np.ones(num_of_f).reshape((-1,1)),X_train))
    X = tf.constant(X_np, dtype=tf.float32, name="X")
    y = tf.constant(Y_train, dtype = tf.float32, name = "y")
    XT = tf.transpose(X)
    
    # define theta for later evaluation
    theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

    with tf.Session() as sess:
        theta_value = theta.eval()
    return theta_value

In [19]:
theta_tf = tf_lin_regress(X_train, Y_train)

In [20]:
theta_tf

array([[0.9994622 ],
       [0.9957907 ],
       [0.49919808],
       [0.20019795]], dtype=float32)

この推定もうまくいっていることが分かる。